In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, cohen_kappa_score
from collections import Counter

In [3]:
df = pd.read_csv("optimal_features.csv")
print("Columns in dataset:", df.columns.tolist())
print("Shape of dataset:", df.shape)

df

Columns in dataset: ['activity', 'trial', 'student', 'window_idx', 'accelerometer_y_var', 'accelerometer_y_range', 'accelerometer_z_min', 'gyroscope_y_range', 'gravity_y_median', 'gravity_z_min', 'gravity_y_max', 'gyroscope_x_range', 'gyroscope_z_range', 'gravity_z_rms', 'gravity_z_range', 'accelerometer_z_iqr', 'gravity_y_rms', 'accelerometer_mag_iqr', 'gravity_y_range', 'gravity_x_max', 'gravity_y_var', 'gyroscope_z_max', 'gyroscope_y_min', 'gyroscope_x_max', 'gravity_x_rms', 'gyroscope_x_iqr', 'gyroscope_z_iqr', 'gyroscope_mag_spec_entropy', 'gyroscope_z_spec_entropy', 'gyroscope_mag_iqr', 'accelerometer_y_mean', 'gyroscope_x_fft_dom_mag', 'gyroscope_z_mean', 'gyroscope_z_median']
Shape of dataset: (160, 34)


,activity,trial,student,window_idx,accelerometer_y_var,accelerometer_y_range,accelerometer_z_min,gyroscope_y_range,gravity_y_median,gravity_z_min,...,gravity_x_rms,gyroscope_x_iqr,gyroscope_z_iqr,gyroscope_mag_spec_entropy,gyroscope_z_spec_entropy,gyroscope_mag_iqr,accelerometer_y_mean,gyroscope_x_fft_dom_mag,gyroscope_z_mean,gyroscope_z_median
0,Running,1,Student1,0,111.290726,59.504999,-15.608119,16.691708,-9.369513,-3.281623,...,2.498269,2.409456,2.342641,2.996621,2.454203,2.094501,-1.803859,0.688195,0.063598,0.097291
1,Running,1,Student1,1,200.973054,66.197945,-16.410546,16.241232,-9.580317,-2.558350,...,2.411305,2.741470,2.785954,3.215959,2.289373,2.163079,-1.400786,0.557416,0.263922,0.624815
2,Running,1,Student1,2,196.090209,63.263831,-19.019897,16.299434,-9.659940,-2.767712,...,2.569917,2.675990,2.865281,3.180685,2.260708,1.961677,-1.235091,0.739358,-0.017637,0.438649
3,Running,1,Student1,3,186.035806,65.169250,-14.555744,15.270794,-9.661414,-2.010538,...,2.090978,2.276336,3.113138,3.280868,2.635878,1.754324,-1.701901,0.544127,0.051824,0.159391
4,Running,2,Student1,0,127.721540,56.712470,-13.716368,11.795485,-9.509268,-3.678784,...,1.940204,2.588911,2.177913,2.957908,2.478304,1.957788,-1.493445,0.684900,0.193169,0.298861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Walking,4,Student1,6,8.101238,16.190116,-6.910718,6.559705,-9.504314,-2.075160,...,1.705484,1.314770,0.902461,2.367397,2.242763,1.140190,-0.354064,0.464157,0.141435,0.145842
156,Walking,4,Student1,7,11.486397,19.590601,-8.400428,6.841438,-9.542667,-2.028722,...,1.640000,0.933684,0.886858,2.344101,2.535850,1.054237,-0.602792,0.401913,-0.063369,0.017672
157,Walking,4,Student1,8,11.340003,18.246516,-6.121468,7.720805,-9.559314,-2.172911,...,1.365229,1.257323,0.876336,2.538802,2.458522,0.949132,-0.227021,0.456699,0.115944,0.254308
158,Walking,4,Student1,9,7.009378,15.007396,-6.723723,5.842770,-9.527957,-2.184663,...,1.742848,1.280315,0.979755,2.195533,2.603396,1.448386,-0.636375,0.504549,0.033303,0.066931


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import (
    accuracy_score, f1_score, cohen_kappa_score,
    confusion_matrix, classification_report
)

# =========================================================
# 0) LOAD & PREPARE DATA
# =========================================================
df = pd.read_csv("optimal_features.csv")

# Label = first column; Features = columns from index 4 onward (skip the first 4 metadata columns)
y = df.iloc[:, 0].astype(str).values
X_all = df.iloc[:, 4:]
# Keep only numeric feature columns
X = X_all.select_dtypes(include=[np.number]).to_numpy()
feature_names = X_all.select_dtypes(include=[np.number]).columns

print("Columns in dataset:", df.columns.tolist())
print("Shape of dataset:", df.shape)
print(f"Using label column: {df.columns[0]!r}")
print(f"Dropped metadata columns: {df.columns[:4].tolist()}")
print(f"Feature columns kept (numeric only): {len(feature_names)}")

print(f"\nTotal samples: {len(X)}")
print(f"Feature count: {X.shape[1]}")
print(f"Class distribution: {Counter(y)}")

# Consistent split for all models
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=33, stratify=y
)

# Utility: unified metrics printer
def evaluate_and_print(y_true, y_pred, model_name, labels_all=None):
    acc = accuracy_score(y_true, y_pred)
    f1m = f1_score(y_true, y_pred, average="macro")
    kappa = cohen_kappa_score(y_true, y_pred)

    if labels_all is None:
        labels_all = np.unique(np.concatenate([y_true, y_pred]))
    cm = confusion_matrix(y_true, y_pred, labels=labels_all)
    cm_df = pd.DataFrame(cm, index=labels_all, columns=labels_all)

    print("\n" + "="*80)
    print(f"{model_name} — Evaluation")
    print(f"Accuracy       : {acc:.4f}")
    print(f"F1-Score Macro : {f1m:.4f}")
    print(f"Cohen's Kappa  : {kappa:.4f}")
    print("\nConfusion Matrix:")
    print(cm_df)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, zero_division=0))
    return acc, f1m, kappa, cm_df

Columns in dataset: ['activity', 'trial', 'student', 'window_idx', 'accelerometer_y_var', 'accelerometer_y_range', 'accelerometer_z_min', 'gyroscope_y_range', 'gravity_y_median', 'gravity_z_min', 'gravity_y_max', 'gyroscope_x_range', 'gyroscope_z_range', 'gravity_z_rms', 'gravity_z_range', 'accelerometer_z_iqr', 'gravity_y_rms', 'accelerometer_mag_iqr', 'gravity_y_range', 'gravity_x_max', 'gravity_y_var', 'gyroscope_z_max', 'gyroscope_y_min', 'gyroscope_x_max', 'gravity_x_rms', 'gyroscope_x_iqr', 'gyroscope_z_iqr', 'gyroscope_mag_spec_entropy', 'gyroscope_z_spec_entropy', 'gyroscope_mag_iqr', 'accelerometer_y_mean', 'gyroscope_x_fft_dom_mag', 'gyroscope_z_mean', 'gyroscope_z_median']
Shape of dataset: (160, 34)
Using label column: 'activity'
Dropped metadata columns: ['activity', 'trial', 'student', 'window_idx']
Feature columns kept (numeric only): 30

Total samples: 160
Feature count: 30
Class distribution: Counter({'Walking': 52, 'Sit': 30, 'Stairs': 30, 'Stand': 30, 'Running': 18})

In [ ]:
# =========================================================
# 1) LOGISTIC REGRESSION (Multinomial)
#    (with scaling)
# =========================================================
scaler_lr = StandardScaler()
X_train_lr = scaler_lr.fit_transform(X_train)
X_test_lr  = scaler_lr.transform(X_test)

lr = LogisticRegression(
    penalty="l2",
    C=1.0,
    multi_class="multinomial",
    solver="lbfgs",
    max_iter=500,
    random_state=33
)

print("\n" + "#"*80)
print("Training: Logistic Regression (Multinomial)")
lr.fit(X_train_lr, y_train)
y_pred_lr = lr.predict(X_test_lr)

acc_lr, f1_lr, kappa_lr, cm_lr = evaluate_and_print(y_test, y_pred_lr, "Logistic Regression (Multinomial)", labels_all=np.unique(y))

# Feature weights (per class)
coef_df = pd.DataFrame(lr.coef_, columns=feature_names, index=lr.classes_)
print("\nTop 5 absolute-weight features per class (Logistic Regression):")
for cls in lr.classes_:
    top5 = coef_df.loc[cls].abs().sort_values(ascending=False).head(5)
    print(f"\nClass: {cls}")
    print(top5)


################################################################################
Training: Logistic Regression (Multinomial)

Logistic Regression (Multinomial) — Evaluation
Accuracy       : 0.9792
F1-Score Macro : 0.9830
Cohen's Kappa  : 0.9731

Confusion Matrix:
         Running  Sit  Stairs  Stand  Walking
Running        5    0       0      0        0
Sit            0    9       0      0        0
Stairs         0    0       9      0        0
Stand          0    0       0      9        0
Walking        0    0       1      0       15

Classification Report:
              precision    recall  f1-score   support

     Running       1.00      1.00      1.00         5
         Sit       1.00      1.00      1.00         9
      Stairs       0.90      1.00      0.95         9
       Stand       1.00      1.00      1.00         9
     Walking       1.00      0.94      0.97        16

    accuracy                           0.98        48
   macro avg       0.98      0.99      0.98        48
w

c:\Users\Silvia\Desktop\tue\.conda\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
# =========================================================
# 2) DECISION TREE
#    (no scaling)
# =========================================================
dt = DecisionTreeClassifier(
    criterion="gini",
    max_depth=None,
    random_state=33
)

print("\n" + "#"*80)
print("Training: Decision Tree")
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

acc_dt, f1_dt, kappa_dt, cm_dt = evaluate_and_print(y_test, y_pred_dt, "Decision Tree", labels_all=np.unique(y))

# Feature importances
importances = pd.Series(dt.feature_importances_, index=feature_names)
top10_dt = importances.sort_values(ascending=False).head(10)
print("\nTop 10 Decision Tree feature importances:")
print(top10_dt)


################################################################################
Training: Decision Tree

Decision Tree — Evaluation
Accuracy       : 0.9583
F1-Score Macro : 0.9580
Cohen's Kappa  : 0.9466

Confusion Matrix:
         Running  Sit  Stairs  Stand  Walking
Running        5    0       0      0        0
Sit            0    9       0      0        0
Stairs         0    0       9      0        0
Stand          0    0       0      9        0
Walking        1    0       1      0       14

Classification Report:
              precision    recall  f1-score   support

     Running       0.83      1.00      0.91         5
         Sit       1.00      1.00      1.00         9
      Stairs       0.90      1.00      0.95         9
       Stand       1.00      1.00      1.00         9
     Walking       1.00      0.88      0.93        16

    accuracy                           0.96        48
   macro avg       0.95      0.97      0.96        48
weighted avg       0.96      0.96      0.

In [ ]:
# =========================================================
# 3) GAUSSIAN NAIVE BAYES
#    (no scaling strictly required; NB is robust to scale)
# =========================================================
gnb = GaussianNB()

print("\n" + "#"*80)
print("Training: Gaussian Naive Bayes")
gnb.fit(X_train, y_train)
y_pred_gnb = gnb.predict(X_test)

acc_gnb, f1_gnb, kappa_gnb, cm_gnb = evaluate_and_print(y_test, y_pred_gnb, "Gaussian Naive Bayes", labels_all=np.unique(y))



################################################################################
Training: Gaussian Naive Bayes

Gaussian Naive Bayes — Evaluation
Accuracy       : 0.9167
F1-Score Macro : 0.9145
Cohen's Kappa  : 0.8926

Confusion Matrix:
         Running  Sit  Stairs  Stand  Walking
Running        5    0       0      0        0
Sit            0    9       0      0        0
Stairs         0    0       9      0        0
Stand          0    3       0      6        0
Walking        0    0       1      0       15

Classification Report:
              precision    recall  f1-score   support

     Running       1.00      1.00      1.00         5
         Sit       0.75      1.00      0.86         9
      Stairs       0.90      1.00      0.95         9
       Stand       1.00      0.67      0.80         9
     Walking       1.00      0.94      0.97        16

    accuracy                           0.92        48
   macro avg       0.93      0.92      0.91        48
weighted avg       0.93    

In [ ]:
# =========================================================
# 4) K-NEAREST NEIGHBORS
#    (with scaling)
# =========================================================
scaler_knn = StandardScaler()
X_train_knn = scaler_knn.fit_transform(X_train)
X_test_knn  = scaler_knn.transform(X_test)

knn = KNeighborsClassifier(
    n_neighbors=5,
    weights="uniform"
)

print("\n" + "#"*80)
print("Training: K-Nearest Neighbors (k=5)")
knn.fit(X_train_knn, y_train)
y_pred_knn = knn.predict(X_test_knn)

acc_knn, f1_knn, kappa_knn, cm_knn = evaluate_and_print(y_test, y_pred_knn, "K-Nearest Neighbors (k=5)", labels_all=np.unique(y))



################################################################################
Training: K-Nearest Neighbors (k=5)

K-Nearest Neighbors (k=5) — Evaluation
Accuracy       : 0.9792
F1-Score Macro : 0.9754
Cohen's Kappa  : 0.9732

Confusion Matrix:
         Running  Sit  Stairs  Stand  Walking
Running        5    0       0      0        0
Sit            0    9       0      0        0
Stairs         0    0       9      0        0
Stand          0    0       0      9        0
Walking        1    0       0      0       15

Classification Report:
              precision    recall  f1-score   support

     Running       0.83      1.00      0.91         5
         Sit       1.00      1.00      1.00         9
      Stairs       1.00      1.00      1.00         9
       Stand       1.00      1.00      1.00         9
     Walking       1.00      0.94      0.97        16

    accuracy                           0.98        48
   macro avg       0.97      0.99      0.98        48
weighted avg     

In [ ]:
# =========================================================
# 5) SUMMARY TABLE (optional but handy)
# =========================================================
summary = pd.DataFrame({
    "Model": [
        "Logistic Regression (Multinomial)",
        "Decision Tree",
        "Gaussian Naive Bayes",
        "K-Nearest Neighbors (k=5)"
    ],
    "Accuracy": [acc_lr, acc_dt, acc_gnb, acc_knn],
    "F1 Macro": [f1_lr, f1_dt, f1_gnb, f1_knn],
    "Cohen's Kappa": [kappa_lr, kappa_dt, kappa_gnb, kappa_knn]
})
print("\n" + "#"*80)
print("Model Comparison Summary")
print(summary.to_string(index=False))


################################################################################
Model Comparison Summary
                            Model  Accuracy  F1 Macro  Cohen's Kappa
Logistic Regression (Multinomial)  0.979167  0.983022       0.973139
                    Decision Tree  0.958333  0.957959       0.946607
             Gaussian Naive Bayes  0.916667  0.914451       0.892557
        K-Nearest Neighbors (k=5)  0.979167  0.975367       0.973199
